# Домашнее задание "Проблема качества данных"

Необходимо запустить практическую часть занятия, и посмотреть самому то, о чём говорили на лекции. По образу практики, попробуйте создать искусственный датасет с лишними столбцами. Целевую метку, при правильной обработке данных, формируйте таким образом, чтобы без затруднений её смогла описать линейная модель. Ориентируйтесь на то, что было показано во время занятия, и каждый шаг описывайте в markdown. Здесь важно видеть ваш ход мысли. Не бойтесь ошибиться или написать не то. Данное задание не имеет какого-то “правильного” решения. Цель - достичь базового понимания проблемы. Чем больше вы фантазируете, тем лучше :) Тем не менее, старайтесь представить те ситуации, которые по-вашему мнению могли бы быть в реальных данных. Успеха!

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LinearRegression

Создаем искуссвтенный датасет с характеристиками и ценой на аквариумы. 

Целевая переменная соответсвенно цена (price).

Параметры:
- артикул товара, vendor_code
- высота, height  
- ширина, width  
- глубина, depth


In [92]:
n_samples = 1000

# a = [4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8] # Какие могут быть диагонали
# b = [32, 64, 128, 256] # варианты встроенной памяти 
# c = [2, 4, 6, 8, 12, 16, 32] # варианты оперативной памяти
# d = [8, 16, 32, 64, 128] # кол-во мегапикселей в камере


vendor_code = np.random.choice(1000, n_samples)
height = np.random.choice(200, n_samples) + 20
width = np.random.choice(200, n_samples) + 20
depth = np.random.choice(200, n_samples) + 20

price = height * width * depth / 1000 +114


In [93]:
data = pd.DataFrame({'vendor_code': vendor_code, 'height': height, 'width': width, 'depth': depth, 'price': price})
data.head()

,vendor_code,height,width,depth,price
0,646,211,127,125,3463.625
1,351,20,200,125,614.000
2,697,158,199,185,5930.770
3,206,44,175,174,1453.800
4,360,175,187,63,2175.675


In [94]:
from sklearn.metrics import mean_absolute_error

X = data[['vendor_code', 'height', 'width', 'depth']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['vendor_code', 'height', 'width', 'depth']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-0.18909212 15.18398077 15.10666793 14.83031512]
Bias: -3438.0451174590125
Error: 549.4868432635466


Получаем очень большую ошибку. Вероятно из-за того, что обучаем модель, учитывая артикул товара, который на ценообразование не влияет. Уберем этот параметр из обучающей выборки. 

In [95]:
X = data[['height', 'width', 'depth']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['height', 'width', 'depth']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [15.23402206 15.09930314 14.79830667]
Bias: -3533.915166558473
Error: 549.5652813312947


Ошибка упала, но незначительно.  Создаем еще одну колонку с параметром "объем".

In [96]:
data['volume'] = data['height'] * data['width'] * data['depth']
data.head(5)

,vendor_code,height,width,depth,price,volume
0,646,211,127,125,3463.625,3349625
1,351,20,200,125,614.000,500000
2,697,158,199,185,5930.770,5816770
3,206,44,175,174,1453.800,1339800
4,360,175,187,63,2175.675,2061675


Обучим выборку только на параметре "объем":

In [97]:
X = data[['volume']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['volume']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [0.001]
Bias: 113.99999999999977
Error: 2.0230572772561573e-13


Ошибка близка к нулю